In [48]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import LocalOutlierFactor
import numpy as np

In [65]:
df = pd.read_excel('input_data.xlsx')
df.head()

,Unnamed: 0,contract_num,product_name,client_id,contract_status,currency_name,duration,country,age,sex,price,insurance_amount,loss_payout_amt,price_usd,insurance_amount_usd,loss_payout_amt_usd
0,0,ТТЕ7227715*****,Страхование путешественников,10161870404,Действует,Российский рубль,10,Беларусь,20,M,1096,2000000,NaN,12.41768,22660.000080,NaN
1,1,ТТЕ7227715*****,Страхование путешественников,1017159879,Действует,Российский рубль,7,Индонезия,55,M,1918,5000000,NaN,21.73094,56650.000199,NaN
2,2,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Действует,Российский рубль,10,Беларусь,61,F,1096,2000000,NaN,12.41768,22660.000080,NaN
3,3,ТТЕ7227715*****,Страхование путешественников,100766461,Действует,Российский рубль,14,Грузия,25,M,1534,2000000,NaN,17.38022,22660.000080,NaN
4,4,ТТЕ7227715*****,Страхование путешественников,10171604758,Действует,Российский рубль,7,Аргентина,33,M,1918,5000000,NaN,21.73094,56650.000199,NaN


In [66]:
df.shape

(3711, 16)

In [67]:
df.isnull().sum()

Unnamed: 0                 0
contract_num               0
product_name               0
client_id                  0
contract_status            0
currency_name              0
duration                   0
country                    0
age                        0
sex                        0
price                      0
insurance_amount           0
loss_payout_amt         3666
price_usd                120
insurance_amount_usd     120
loss_payout_amt_usd     3668
dtype: int64

In [68]:
dfle = df.reset_index().drop("index",axis=1).apply(lambda series: pd.Series (
LabelEncoder().fit_transform(series[series.notnull()]),
index=series[series.notnull()].index))

In [69]:
imputer = KNNImputer(n_neighbors=4)

In [70]:
dfle_imp = imputer.fit_transform(dfle)

In [71]:
dfnorm = pd.DataFrame(dfle_imp,columns=dfle.columns)

In [72]:
dfnorm.isnull().sum()

Unnamed: 0              0
contract_num            0
product_name            0
client_id               0
contract_status         0
currency_name           0
duration                0
country                 0
age                     0
sex                     0
price                   0
insurance_amount        0
loss_payout_amt         0
price_usd               0
insurance_amount_usd    0
loss_payout_amt_usd     0
dtype: int64

In [73]:
dfamomal = dfnorm.drop(['Unnamed: 0','contract_num','client_id','product_name','currency_name','country','sex','contract_status'],axis=1)

In [74]:
clf = LocalOutlierFactor(n_neighbors=5)

In [75]:
pred = clf.fit_predict(dfamomal)

In [76]:
dfamomal[pred==-1].mean() - dfamomal.mean()

duration               -0.471830
age                     0.449696
price                  -0.884218
insurance_amount       -0.078382
loss_payout_amt        -0.222547
price_usd              -1.029933
insurance_amount_usd   -0.107935
loss_payout_amt_usd    -0.060265
dtype: float64

In [102]:
q99,q1 = np.percentile(dfamomal.duration,[99,1])
print(q99,q1)

22.0 0.0


In [103]:
q99,q1 = np.percentile(dfamomal.age,[99,1])
print(q99,q1)
q99,q1 = np.percentile(dfamomal.price,[99,1])
print(q99,q1)
q98,q2 = np.percentile(dfamomal.insurance_amount,[98,2])
print(q99,q1)
q98,q2 = np.percentile(dfamomal.loss_payout_amt,[98,2])
print(q99,q1)
q98,q2 = np.percentile(dfamomal.price_usd,[98,2])
print(q99,q1)
q98,q2 = np.percentile(dfamomal.insurance_amount_usd,[98,2])
print(q99,q1)


45.0 1.0
44.0 2.0
44.0 2.0
44.0 2.0
44.0 2.0
44.0 2.0


In [104]:
df_anomal_a = dfnorm[(dfnorm.age<q1)|(dfnorm.age>q99)]

In [105]:
df_anomal_p = dfnorm[(dfnorm.price<q1)|(dfnorm.price>q99)]

In [106]:
df_anomal_d = dfnorm[(dfnorm.duration<q1)|(dfnorm.duration>q99)]

In [107]:
df_anomal_i = dfnorm[(dfnorm.insurance_amount<q1)|(dfnorm.insurance_amount>q99)]

In [108]:
df_anomal_l = dfnorm[(dfnorm.loss_payout_amt<q1)|(dfnorm.loss_payout_amt>q99)]

In [109]:
df_anomal_pu = dfnorm[(dfnorm.price_usd<q1)|(dfnorm.price_usd>q99)]

In [110]:
df_anomal_iu = dfnorm[(dfnorm.insurance_amount_usd<q1)|(dfnorm.insurance_amount_usd>q99)]

In [111]:
df_anomal = pd.concat([df_anomal_a,df_anomal_p,df_anomal_d,df_anomal_i,df_anomal_l,df_anomal_pu,df_anomal_iu])

In [112]:
df_anomal

,Unnamed: 0,contract_num,product_name,client_id,contract_status,currency_name,duration,country,age,sex,price,insurance_amount,loss_payout_amt,price_usd,insurance_amount_usd,loss_payout_amt_usd
0,0.0,74.0,0.0,3372.0,0.0,1.0,1.0,3.0,0.0,1.0,18.0,3.0,1.75,1.0,1.0,4.50
51,51.0,25.0,0.0,519.0,0.0,1.0,5.0,7.0,1.0,1.0,19.0,3.0,2.50,2.0,1.0,4.50
56,56.0,40.0,1.0,534.0,0.0,0.0,3.0,14.0,51.0,1.0,5.0,1.0,2.50,2.0,1.0,4.50
83,83.0,31.0,0.0,3548.0,0.0,1.0,5.0,9.0,46.0,0.0,29.0,5.0,4.00,11.0,3.0,4.50
100,100.0,40.0,0.0,1804.0,0.0,1.0,1.0,17.0,1.0,0.0,20.0,4.0,1.00,3.0,2.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3696,3696.0,54.0,0.0,3357.0,0.0,1.0,1.0,11.0,6.0,1.0,18.0,3.0,9.25,1.0,1.0,9.25
3697,3697.0,33.0,0.0,3360.0,0.0,1.0,1.0,11.0,14.0,0.0,18.0,3.0,9.25,1.0,1.0,9.25
3699,3699.0,7.0,0.0,762.0,0.0,1.0,1.0,3.0,25.0,1.0,18.0,3.0,11.25,1.0,1.0,11.25
3702,3702.0,40.0,0.0,3369.0,0.0,1.0,1.0,3.0,14.0,1.0,18.0,3.0,9.25,1.0,1.0,9.25


In [113]:
df_anomal['anomalies'] = 1

In [114]:
df_anomal

,Unnamed: 0,contract_num,product_name,client_id,contract_status,currency_name,duration,country,age,sex,price,insurance_amount,loss_payout_amt,price_usd,insurance_amount_usd,loss_payout_amt_usd,anomalies
0,0.0,74.0,0.0,3372.0,0.0,1.0,1.0,3.0,0.0,1.0,18.0,3.0,1.75,1.0,1.0,4.50,1
51,51.0,25.0,0.0,519.0,0.0,1.0,5.0,7.0,1.0,1.0,19.0,3.0,2.50,2.0,1.0,4.50,1
56,56.0,40.0,1.0,534.0,0.0,0.0,3.0,14.0,51.0,1.0,5.0,1.0,2.50,2.0,1.0,4.50,1
83,83.0,31.0,0.0,3548.0,0.0,1.0,5.0,9.0,46.0,0.0,29.0,5.0,4.00,11.0,3.0,4.50,1
100,100.0,40.0,0.0,1804.0,0.0,1.0,1.0,17.0,1.0,0.0,20.0,4.0,1.00,3.0,2.0,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3696,3696.0,54.0,0.0,3357.0,0.0,1.0,1.0,11.0,6.0,1.0,18.0,3.0,9.25,1.0,1.0,9.25,1
3697,3697.0,33.0,0.0,3360.0,0.0,1.0,1.0,11.0,14.0,0.0,18.0,3.0,9.25,1.0,1.0,9.25,1
3699,3699.0,7.0,0.0,762.0,0.0,1.0,1.0,3.0,25.0,1.0,18.0,3.0,11.25,1.0,1.0,11.25,1
3702,3702.0,40.0,0.0,3369.0,0.0,1.0,1.0,3.0,14.0,1.0,18.0,3.0,9.25,1.0,1.0,9.25,1


In [121]:
df_full = pd.merge(df_anomal,dfnorm,left_on='Unnamed: 0', right_on='Unnamed: 0', how='outer')

In [122]:
df_full

,Unnamed: 0,contract_num_x,product_name_x,client_id_x,contract_status_x,currency_name_x,duration_x,country_x,age_x,sex_x,...,duration_y,country_y,age_y,sex_y,price_y,insurance_amount_y,loss_payout_amt_y,price_usd_y,insurance_amount_usd_y,loss_payout_amt_usd_y
0,0.0,74.0,0.0,3372.0,0.0,1.0,1.0,3.0,0.0,1.0,...,1.0,3.0,0.0,1.0,18.0,3.0,1.75,1.0,1.0,4.5
1,0.0,74.0,0.0,3372.0,0.0,1.0,1.0,3.0,0.0,1.0,...,1.0,3.0,0.0,1.0,18.0,3.0,1.75,1.0,1.0,4.5
2,0.0,74.0,0.0,3372.0,0.0,1.0,1.0,3.0,0.0,1.0,...,1.0,3.0,0.0,1.0,18.0,3.0,1.75,1.0,1.0,4.5
3,0.0,74.0,0.0,3372.0,0.0,1.0,1.0,3.0,0.0,1.0,...,1.0,3.0,0.0,1.0,18.0,3.0,1.75,1.0,1.0,4.5
4,0.0,74.0,0.0,3372.0,0.0,1.0,1.0,3.0,0.0,1.0,...,1.0,3.0,0.0,1.0,18.0,3.0,1.75,1.0,1.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,3703.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,17.0,12.0,1.0,27.0,4.0,11.00,9.0,2.0,11.0
5944,3705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,17.0,10.0,1.0,27.0,4.0,11.00,9.0,2.0,11.0
5945,3706.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,17.0,2.0,0.0,27.0,4.0,11.00,9.0,2.0,11.0
5946,3708.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,17.0,23.0,1.0,27.0,4.0,8.50,9.0,2.0,8.5


In [123]:
df_full['anomalies'].unique

<bound method Series.unique of 0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
5943    NaN
5944    NaN
5945    NaN
5946    NaN
5947    NaN
Name: anomalies, Length: 5948, dtype: float64>

In [125]:
df_full['anomalies']=df_full['anomalies'].fillna(0)

In [126]:
dfle = df_full.reset_index().drop("index",axis=1).apply(lambda series: pd.Series (
LabelEncoder().fit_transform(series[series.notnull()]),
index=series[series.notnull()].index))

In [127]:
imputer = KNNImputer(n_neighbors=4)

In [128]:
dfle_imp = imputer.fit_transform(dfle)
dfnorm_full = pd.DataFrame(dfle_imp,columns=dfle.columns)

In [129]:
dfnorm_full.isnull().sum()

Unnamed: 0                0
contract_num_x            0
product_name_x            0
client_id_x               0
contract_status_x         0
currency_name_x           0
duration_x                0
country_x                 0
age_x                     0
sex_x                     0
price_x                   0
insurance_amount_x        0
loss_payout_amt_x         0
price_usd_x               0
insurance_amount_usd_x    0
loss_payout_amt_usd_x     0
anomalies                 0
contract_num_y            0
product_name_y            0
client_id_y               0
contract_status_y         0
currency_name_y           0
duration_y                0
country_y                 0
age_y                     0
sex_y                     0
price_y                   0
insurance_amount_y        0
loss_payout_amt_y         0
price_usd_y               0
insurance_amount_usd_y    0
loss_payout_amt_usd_y     0
dtype: int64

In [130]:
dfnorm_full = dfnorm_full.drop(['Unnamed: 0'],axis=1)

In [131]:
dfnorm_full.to_excel (r'/Users/matvejterebilenko/Downloads/Новая таблица-3.xlsx', index= False )